In [ ]:
import pandas as pd
import json

In [ ]:
csv_path = '../data/csv/data_2025_12_24-14h_00m.csv'
df = pd.read_csv(csv_path)

### Estadisticas basicas

In [ ]:
print("---1. VIsta previa ---")
print(df.head(1))

In [ ]:
print("---2. Dimensiones ---")
print(f"Filas: {df.shape[0]}, COlumnas: {df.shape[1]}")

In [ ]:
print("---3. Tipos de datos ---")
print(df.info())

In [ ]:
print("---4. Conteo de autos unicos")
unique_cars = df['ID_Auto'].nunique()
print(f"Total de autos unicos: {unique_cars}")

In [ ]:
print("---5. Estadisticas Rapidas ---")
print(df[['Precio', 'Mensualidad', 'Tasa', 'Enganche_Simulado', 'Enganche_Min', 'Enganche_Max']].describe())

### Encontrando Insights

In [ ]:
# Transform data...
df_fin = pd.read_csv(csv_path)

#Diccionario {'auto_id':'Nombre del auto'}
autos_dict = {}
with open('../data/dataset_autos.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        car = json.loads(line)
        name = car['slug'].split('/')[-1].replace('-', ' ').title()
        autos_dict[car['id']] = name

#VLOOKUP by ID
df_fin['Nombre'] = df_fin['ID_Auto'].astype(str).map(autos_dict)

# Separate name into ['Brand', 'Model', 'Version', 'Type', 'Year']
df_fin[['Brand', 'Model', 'Version', 'Type', 'Year']] = (
    df_fin['Nombre']
    .str.strip()
    .str.split(' ', expand=True)
)
df_fin = df_fin.drop(columns=['Nombre'])

In [ ]:
# The Rate Hunter
print("--- TOP 5 AUTOS CON LAS MENSUALIDADES MAS BAJA (Plazo 48) ---")
df_48 = df_fin[df_fin['Plazo'] == 48].copy()

best_rates = df_48.sort_values('Mensualidad').head(5)

print(best_rates[['Brand', 'Model', 'Type', 'Year', 'Precio', 'Tasa', 'Mensualidad']])

In [ ]:
print("--- Costo Total vs Contado ---")

#Pago total por el vehiculo.
df_fin['Total_a_Pagar'] = (df_fin['Mensualidad'] * df_fin['Plazo']) + df_fin['Enganche_Simulado'] 

#Interes total y porcentaje extra respecto al Precio del vehiculo
df_fin['Interes_Total'] = df_fin['Total_a_Pagar'] - df_fin['Precio']
df_fin['Porcentaje_Extra'] = (df_fin['Interes_Total'] * 100) / df_fin['Precio']

In [ ]:
# Smart Buys
# Autos donde el financiamiento sea menor al 40% del valor del auto con un plazo de 48 meses
smart_buys = df_fin[
    (df_fin['Plazo'] == 48) #&
    #(df_fin['Porcentaje_Extra'] < 45) # Limite porcentual maximo de financiamiento respecto al precio del auto
].sort_values('Porcentaje_Extra', ascending=True)

if not smart_buys.empty:
    print(smart_buys[['ID_Auto', 'Precio', 'Plazo', 'Mensualidad', 'Total_a_Pagar', 'Porcentaje_Extra']].head(10))
else:
    print("No hay autos con financiamiento en este porcentaje")

In [ ]:

worst_deals = df_fin[df_fin['Plazo'] == 60].sort_values('Porcentaje_Extra', ascending=False).head(10)
print("Los 10 autos donde pagas más intereses proporcionales (a 60 meses):")
print(worst_deals[['Brand', 'Model', 'Plazo', 'Precio', 'Total_a_Pagar', 'Porcentaje_Extra']])

In [ ]:
#The cashflow King
print("\n--- AUTOS CON VALOR MAYOR A 300k CON MENSUALIDAD MENORES A 10K ---")

luxury_cheap = df_fin[
    (df_fin['Precio'] > 300000) &
    (df_fin['Mensualidad'] < 10000)
].sort_values('Porcentaje_Extra', ascending=True).head()



print(luxury_cheap[['ID_Auto', 'Brand', 'Model', 'Type', 'Year', 'Precio', 'Tasa', 'Plazo', 'Mensualidad', 'Total_a_Pagar', 'Porcentaje_Extra']])

In [ ]:
# Filtramos específicamente ese Nissan Sentra usando tu DataFrame ya cargado
# (Asegúrate de tener df_fin cargado del paso anterior)

sentra_id = 1003531 # Lo vi en tu tabla anterior del análisis "Premium"
sentra_real = df_fin[
    (df_fin['ID_Auto'] == sentra_id) & 
    (df_fin['Plazo'] == 72)
].copy()

# Calculamos la realidad
sentra_real['Total_a_Pagar'] = (sentra_real['Mensualidad'] * sentra_real['Plazo']) + sentra_real['Enganche_Simulado']
sentra_real['Interes_Pagado'] = sentra_real['Total_a_Pagar'] - sentra_real['Precio']

print("--- 💸 LA VERDAD DEL NISSAN SENTRA 2023 ---")
print(f"Precio de Contado:      ${sentra_real['Precio'].values[0]:,.2f}")
print(f"Enganche Mínimo (16%):  ${sentra_real['Enganche_Simulado'].values[0]:,.2f}")
print(f"Tu Mensualidad Real:    ${sentra_real['Mensualidad'].values[0]:,.2f}")
print(f"Total a pagar (6 años): ${sentra_real['Total_a_Pagar'].values[0]:,.2f}")
print(f"Intereses Extra:        ${sentra_real['Interes_Pagado'].values[0]:,.2f}")
print(f"Intere Porcentual Extra: {sentra_real['Porcentaje_Extra'].values[0]:,.2f}%")